In [41]:
import numpy as np
from colors import *

In [28]:
SCORING = {
    "SINGLE": 100,
    "DOUBLE": 300,
    "TRIPLE": 500,
    "QUAD": 800,
    "TSPIN_MINI": 100,
    "TSPIN": 400,
    "TSPIN_MINI_SINGLE": 200,
    "TSPIN_SINGLE": 800,
    "TSPIN_MINI_DOUBLE": 400,
    "TSPIN_DOUBLE": 1200,
    "TSPIN_TRIPLE": 1600,
    "TSPIN_QUAD": 2600,
    "BACKTOBACK_MULTIPLIER": 1.5,
    "COMBO": 50,
    "ALL_CLEAR": 3500,
    "SOFTDROP": 1,
    "HARDDROP": 2
}

In [108]:
class PieceData():
    def __init__(self, data, num_rot, start_x_position):
        self.num_rot = num_rot
        self.start_x_position = start_x_position
        
        self.data = []
        self.width = []
        self.height = []
        for i in range(num_rot):
            cells=np.rot90(data, i)
            self.data.append(cells)
            self.width.append(np.argwhere((cells == 1).any(axis=0)).max() + 1)
            self.height.append(np.argwhere((cells == 1).any(axis=1)).max() + 1)
        # self.data = np.array(self.data)

    def __str__(self):
        return '\n'.join(map(lambda x: str(x), self.data))
    def __repr__(self):
        return self.__str__()
        
        
piece_data = []

# 0: t-piece
piece_data.append(PieceData(np.array([(0,1,0),(1,1,1),(0,0,0)]), 4, 3))
# 1: l-piece
piece_data.append(PieceData(np.array([(0,0,1),(1,1,1), (0,0,0)]), 4, 3))
# 2: j-piece
piece_data.append(PieceData(np.array([(1,0,0),(1,1,1),(0,0,0)]), 4, 3))
# 3: o-piece
piece_data.append(PieceData(np.array([(0,1,1,0),(0,1,1,0),(0,0,0,0)]), 1, 4))
# 4: i-piece
piece_data.append(PieceData(np.array([(0,0,0,0),(1,1,1,1),(0,0,0,0),(0,0,0,0)]), 4, 3))
# 5: s-piece
piece_data.append(PieceData(np.array([(0,1,1),(1,1,0),(0,0,0)]), 4, 3))
# 6: z-piece
piece_data.append(PieceData(np.array([(1,1,0),(0,1,1),(0,0,0)]), 4, 3))


kicks = {
    # (kick_from, kick_to): (offset1, offset2, offset3, offset4)
    (0, 1): ((+0, -1), (-1, -1), (+2, +0), (+2, -1)),  # 0 -> R | CW
    (0, 3): ((+0, +1), (-1, +1), (+2, +0), (+2, +1)),  # 0 -> L | CCW
    (1, 0): ((+0, +1), (+1, +1), (-2, +0), (-2, +1)),  # R -> 0 | CCW
    (1, 2): ((+0, +1), (+1, +1), (-2, +0), (-2, +1)),  # R -> 2 | CW
    (2, 1): ((+0, -1), (-1, -1), (+2, +0), (+2, -1)),  # 2 -> R | CCW
    (2, 3): ((+0, +1), (-1, +1), (+2, +0), (+2, +1)),  # 2 -> L | CW
    (3, 0): ((+0, -1), (+1, -1), (-2, +0), (-2, -1)),  # L -> 0 | CW
    (3, 2): ((+0, -1), (+1, -1), (-2, +0), (-2, -1)),  # L -> 2 | CCW
}

i_kicks = {
    (0, 1): ((+0, -2), (+0, +1), (+1, -2), (-2, +1)),  # 0 -> R | CW
    (0, 3): ((+0, -1), (+0, +2), (-2, -1), (+1, +2)),  # 0 -> L | CCW
    (1, 0): ((+0, +2), (+0, -1), (-1, +2), (+2, -1)),  # R -> 0 | CCW
    (1, 2): ((+0, -1), (+0, +2), (-2, -1), (+1, +2)),  # R -> 2 | CW
    (2, 1): ((+0, +1), (+0, -2), (+2, +1), (-1, +2)),  # 2 -> R | CCW
    (2, 3): ((+0, +2), (+0, -1), (-1, +2), (+2, -1)),  # 2 -> L | CW
    (3, 0): ((+0, +1), (+0, -2), (+2, +1), (-1, -2)),  # L -> 0 | CW
    (3, 2): ((+0, -2), (+0, +1), (+1, -2), (-2, +1)),  # L -> 2 | CCW
}

In [106]:
class Piece():
    def __init__(self, ptype, pos=None):
        self.pos = pos if pos is not None else [0, piece_data[ptype].start_x_position]
        self.ptype = ptype
        self.pdata = piece_data[ptype]
    
    def __str__(self):
        return str(self.pdata)
    def __repr__(self):
        return self.__str__()
    
    def clone(self, ptype=None, pos=None):
        if ptype is None:
            ptype = self.ptype
        if pos is None:
            pos = self.pos
        
        return Piece(ptype, pos)
        
# def drop_piece_on_field(P, F):

p1 = Piece(4)
p1.pdata.data[3]


# for i, pd in enumerate(p1.pdata.data):
#     print(i)
#     print(pd)
#     print()

# p2 = Piece(0)

# id(p1.pdata) == id(p2.pdata)

array([[0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0],
       [0, 0, 1, 0]])

In [126]:
class Field():
    def __init__(self, shape=(20, 10), data=None):
        if data is None:
            self.field_data = np.zeros(shape) # np.arange(10*20).reshape((20,10))
        else:
            self.field_data = data

        self.width = shape[1]
        self.height = shape[0]
        
    def is_collision(self, piece, rot, pos=None):
        """ Returns whether the given piece collides with the current field. """
        # check if collision between shape and existing play field
        # if 'pos' is not passed, it will use the piece's position
        (py, px, pw, ph) = self._extract_piece_info(piece, rot, pos)
        # pw = piece.pdata.width[rot]
        # ph = piece.pdata.height[rot]
        
        if py + ph > self.height:
            return True
        if px + pw > self.width:
            return True

        # print('px: ', px, 'py: ', py,  'pw', pw, 'ph', ph)
        # Source: https://github.com/vprinsen/numpy_tetris/blob/master/tetris.py
        temp_field = np.zeros(self.field_data.shape)
        # copy piece data at the piece's position to the temporary field
        temp_field[py:py + ph, px:px + pw] = piece.pdata.data[rot]
        # print('temp field')
        # print(np.logical_or(PF, self.field_data) * 1)
        # multiply field with temporary field and check if there is a position,
        # where both the cells are set (= 1). only if both are 1, the product is non-zero
        
        return (np.count_nonzero(temp_field*self.field_data)) > 0

        #  https://stackoverflow.com/questions/31407624/colission-testing-with-numpy
        #  return (B[x:x+3,y:y+3] * T).sum() == 0
        #  return (B[x:,y:][:3,:3] * T).sum() == 0
    
    # SLOWER
    def is_collision2(self, piece, rot, pos=None):
        py = piece.pos[0] if pos is None else pos[0]
        px = piece.pos[1] if pos is None else pos[1]
        pw = piece.pdata.width[rot]
        ph = piece.pdata.height[rot]
        
        return (self.field_data[py:py + ph, px:px + pw] * piece.pdata.data[rot]).sum() != 0
        
    def get_follow_states(self, piece):
        """ Generates all possible next states that follow from the current state of the field. """
        next_states = []
        
        # rotate the piece in all possible directions
        for rot in range(piece.pdata.num_rot):
            p_data = piece.pdata.data[rot]
            
            width_diff = self.width - p_data.shape[1]
            
            # move the piece to all possible positions
            for px in range(width_diff + 1):
                py = self.drop_piece(piece, rot, [0, px])
                
                dropped_pos = [py, px]
                
                print(f'DROPPED {piece.pdata.data[rot]} TO [{py}, {px}] WITH ROT = {rot}, KICKFLIPPING:')
                self._print_board(self.add_piece_to_board(piece, rot, dropped_pos))
                
                # try kick/flip with all possible rotations
                for kick_rot in range(piece.pdata.num_rot):
                    (kick_succ, kick_result) = self._rotate(piece, rot, rot - kick_rot, dropped_pos)
                    
                    if kick_succ and self.is_collision(piece, kick_rot, [py - 1, px]):
                        print(f'SUCESSFULLY KICKFLIPPED: {kick_result}')
                        
                        self._print_board(self.add_piece_to_board(piece, kick_result[2], [kick_result[0], kick_result[1]])) 
                        
                        # possible state
                        next_states.append(Field(data=self.drop_piece_on_field(piece, kick_result[2], [kick_result[0], kick_result[1]])))
                
                # new_field = self.drop_piece_on_field(piece, rot, [0, px])
                # next_states.append(Field(data=new_field))
                
        return next_states

    def _extract_piece_info(self, piece, rot, pos=None):
        py = piece.pos[0] if pos is None else pos[0]
        px = piece.pos[1] if pos is None else pos[1]
        pw = piece.pdata.data[rot].shape[1]
        ph = piece.pdata.data[rot].shape[0] 
        
        return (py, px, pw, ph)
    
    
    def _rotate(self, piece, rot, rot_offset, pos=None):
        """Tries to kick flip/spin the given piece and returns if the rotation worked and the new piece infos, as a tuple."""
        (py, px, pw, ph) = self._extract_piece_info(piece, rot, pos)
        to_rot = (rot + rot_offset) % piece.pdata.num_rot
        
        # if it does not collide, just rotate it
        if not self.is_collision(piece, rot, pos):
            return (True, (py, px, to_rot))
        
        # otherwise try to kick it
        spin_table = i_kicks if piece.ptype == 4 else kicks
        
        return self._kick_piece(table, piece, rot, to_rot, pos)
        
#         for move in spin_table:
#             _px = px + move[0]
#             _py = py + move[1]
#             _rot = (rot + move[2]) % piece.pdata.num_rot

#             if not self.is_collision(piece, _rot, [_py, _px]):
#                 print(f'===== KICKFLIPPING WITH MOVE: {move}, returning {(_py, _px, _rot)}, input = {(py, px, rot)}')
#                 return (True, (_py, _px, _rot))

        # return (False, None)
    def _kick_piece(self, table, piece, rot, to_rot, pos=None):
        (py, px, pw, ph) = self._extract_piece_info(piece, rot, pos)
        
        if not (rot, to_rot) in table:
            return
        
        for x, y in table[rot, to_rot]:
            # for each offset, test if it's valid
            if not self.is_collision(piece, rot, [py + y, px + x]):
                # if it's vlaid, kick it and break
                return (True, (py + y, px + x, to_rot))
        
        return (False, None)
        
    
    def drop_piece(self, piece, rot, pos=None):
        """Tries to drop the given piece on this field and returns the height it landed on."""
        (py, px, pw, ph) = self._extract_piece_info(piece, rot, pos)
        
        while not self.is_collision(piece, rot, [py, px]):
            py += 1

        return py - 1
    
    def add_piece_to_board(self, piece, rot, pos=None):
        (py, px, pw, ph) = self._extract_piece_info(piece, rot, pos)
        
        # copy piece data at the piece's position to a temporary field
        temp_field = np.zeros(self.field_data.shape)
        temp_field[py:py + ph, px:px + pw] = piece.pdata.data[rot]
        
        return np.logical_or(temp_field, self.field_data) * 1
        
    
    def drop_piece_on_field(self, piece, rot, pos=None):
        """ Drops the given piece on this field, returning the newly generated field."""
        (py, px, pw, ph) = self._extract_piece_info(piece, rot, pos)
        
        py = self.drop_piece(piece, rot, pos)
        
        return self.add_piece_to_board(piece, rot, [py, px])
    
    
    def drop_piece_on_field_(self, piece, rot, pos=None):
        """ Drops the given piece on this field in-place."""
        self.field_data = self.drop_piece_on_field(piece, rot, pos)
    
    def _print_board(self, board):
        print(str(board))
    
    def __str__(self):
        return str(self.field_data)
    
    def __repr__(self):
        return self.__str__()

In [127]:
f = Field()
# f.field_data[18:, :] = 1
# f.field_data[17,1] = 1
# f.field_data[17,2] = 1
# f.field_data[17,5] = 1
# f.field_data[17,6] = 1
# print(f)

p = Piece(0, [0,3])
print(p)

# f.get_follow_states(p)
f.drop_piece_on_field_(p, 0)
# f.field_data[-1]

# f.field_data[1,1]

p2 = Piece(4)

f.get_follow_states(p2)

[[0 1 0]
 [1 1 1]
 [0 0 0]]
[[0 1 0]
 [1 1 0]
 [0 1 0]]
[[0 0 0]
 [1 1 1]
 [0 1 0]]
[[0 1 0]
 [0 1 1]
 [0 1 0]]
DROPPED [[0 0 0 0]
 [1 1 1 1]
 [0 0 0 0]
 [0 0 0 0]] TO [16, 0] WITH ROT = 0, KICKFLIPPING:
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [1 1 1 1 1 0 0 0 0 0]
 [0 0 0 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]]
DROPPED [[0 0 0 0]
 [1 1 1 1]
 [0 0 0 0]
 [0 0 0 0]] TO [15, 1] WITH ROT = 0, KICKFLIPPING:
[[0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0

[[[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 1 0 0 0 0 0 0]
  [0 0 0 1 1 0 0 0 0 0]
  [0 0 0 1 1 1 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]],
 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 0 1 0 0 0 0]
  [0 0 0 0 1 1 0 0 0 0]
  [0 0 0 1 1 1 0 0 0 0]
  [0 0 0 0 0 0 0 0 0 0]],
 [[0 0 0 0 0 0 0 0 0 0]
  [0 0 0 0 0

%timeit np.multiply(a, b)
1.65 µs ± 67.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.67 µs ± 35.9 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

%timeit a * b
1.64 µs ± 47.6 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.65 µs ± 39 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


%timeit (np.add(a, b))
1.46 µs ± 68.7 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)
1.47 µs ± 58.3 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)

%timeit (np.count_nonzero(a*b))
2.99 µs ± 88.3 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
3.22 µs ± 169 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
2.91 µs ± 40.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


